In [1]:
%run util.py

/home/jzh/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

# Loading JSON data
with open(data_folder + 'jam_ref_data.json', 'r') as json_file:
    jam_ref_data = json.load(json_file)
    
with open(data_folder + 'jam_test_data.json', 'r') as json_file:
    jam_test_data = json.load(json_file)

### Quantize the features by k-means clustering

In [4]:
### Quantize the features by k-means clustering 

import numpy as np

loc_list = []
length_list = []
numPts_list = []
speed_list = []

ref_loc_list = []
ref_length_list = []
ref_numPts_list = []
ref_speed_list = []
for key in jam_ref_data.keys():
    ref_loc_list.append(jam_ref_data[key]['longitude_latitude'])
    loc_list.append(jam_ref_data[key]['longitude_latitude'])
    
    ref_length_list.append(jam_ref_data[key]['length'])
    length_list.append(jam_ref_data[key]['length'])
    
    ref_numPts_list.append(float(jam_ref_data[key]['numPts']))
    numPts_list.append(float(jam_ref_data[key]['numPts']))
    
    ref_speed_list.append(jam_ref_data[key]['speed'])
    speed_list.append(jam_ref_data[key]['speed'])


test_loc_list = []
test_length_list = []
test_numPts_list = []
test_speed_list = []
for key in jam_test_data.keys():
    test_loc_list.append(jam_test_data[key]['longitude_latitude'])
    loc_list.append(jam_test_data[key]['longitude_latitude'])
    
    test_length_list.append(jam_test_data[key]['length'])
    length_list.append(jam_test_data[key]['length'])
    
    test_numPts_list.append(float(jam_test_data[key]['numPts']))
    numPts_list.append(float(jam_test_data[key]['numPts']))
    
    test_speed_list.append(jam_test_data[key]['speed'])
    speed_list.append(jam_test_data[key]['speed'])

loc_data = loc_list
length_data = np.array(length_list)
numPts_data = np.array(numPts_list)
speed_data = np.array(speed_list)

In [5]:
len(loc_data), len(ref_loc_list), len(test_loc_list)

(6586, 2830, 3756)

### Quantization levels: loc_k = 2, length_k = 2, numPts_k = 1, speed_k = 2

In [6]:
loc_data = [list(np.reshape(loc_data[i], np.size(loc_data[i]))) for i in range(len(loc_data))]

from scipy.cluster.vq import kmeans2
from ClusterAlg import DF, KMedians

loc_k = 2
num_core = 12
loc_label, loc_centroid, distToClusterCenter = KMedians(loc_data, loc_k, num_core, DF)

In [7]:
length_k = 2
length_centroid, length_label = kmeans2(length_data, length_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

numPts_k = 1
numPts_centroid, numPts_label = kmeans2(numPts_data, numPts_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

speed_k = 2
speed_centroid, speed_label = kmeans2(speed_data, speed_k, iter=300, thresh=1e-06, minit='random', missing='warn', check_finite=True)

In [8]:
distToClusterCenter, loc_label, loc_centroid

([2090.8648379686165,
  572.255493462783,
  1511.826516561467,
  1316.9132342546513,
  758.4585473911834,
  599.5921401055803,
  631.8454866955852,
  431.5385946947995,
  1650.2800757758066,
  599.5921401055803,
  631.8454866955852,
  1868.5881087666355,
  599.5921401055803,
  631.8454866955852,
  991.804705000757,
  3306.0813874006353,
  1554.503838752726,
  1495.3600039004177,
  1693.8082888937213,
  1299.5570086954042,
  1661.3592732423322,
  1069.5346672639807,
  153.94619565295974,
  1593.227506728265,
  744.1162916329997,
  856.2709961745086,
  599.5921401055803,
  631.8454866955852,
  736.7582823578451,
  554.1668162273843,
  1710.4443295850622,
  1495.3600039004177,
  363.555695726105,
  1152.1783959079776,
  631.8454866955852,
  977.4356704759158,
  363.555695726105,
  1464.8289517275775,
  837.8447810957348,
  783.7144935575734,
  599.5921401055803,
  631.8454866955852,
  363.555695726105,
  922.6829521087968,
  2531.3655379721927,
  599.5921401055803,
  281.6497477155183,
  

In [9]:
ref_loc_label = loc_label[0:np.size(np.array(ref_loc_list),0)]
test_loc_label = loc_label[np.size(np.array(ref_loc_list),0):np.size(loc_data,0)]

ref_length_label = length_label[0:np.size(np.array(ref_length_list),0)]
test_length_label = length_label[np.size(np.array(ref_length_list),0):np.size(length_data,0)]

ref_numPts_label = numPts_label[0:np.size(np.array(ref_numPts_list),0)]
test_numPts_label = numPts_label[np.size(np.array(ref_numPts_list),0):np.size(numPts_data,0)]

ref_speed_label = speed_label[0:np.size(np.array(ref_speed_list),0)]
test_speed_label = speed_label[np.size(np.array(ref_speed_list),0):np.size(speed_data,0)]

In [10]:
len(ref_loc_label), len(test_loc_label), len(ref_length_label), len(test_length_label), \
len(ref_numPts_label), len(test_numPts_label), len(ref_speed_label), len(test_speed_label)

(2830, 3756, 2830, 3756, 2830, 3756, 2830, 3756)

In [11]:
quantized_jam_ref_data = {}

i = 0
for key in jam_ref_data.keys():
    data = {'uuid': jam_ref_data[key]['uuid'],
            'longitude_latitude': jam_ref_data[key]['longitude_latitude'],
            'length': jam_ref_data[key]['length'],
            'numPts': float(jam_ref_data[key]['numPts']),
            'speed': jam_ref_data[key]['speed'],
            'startTime': jam_ref_data[key]['startTime'],
            'loc_label': float(ref_loc_label[i]),
            'length_label': float(ref_length_label[i]),
            'numPts_label': float(ref_numPts_label[i]),
            'speed_label': float(ref_speed_label[i])}
    quantized_jam_ref_data[key] = data
    i += 1
    
# Writing JSON data
with open(data_folder + 'quantized_jam_ref_data.json', 'w') as json_file:
    json.dump(quantized_jam_ref_data, json_file)

In [12]:
quantized_jam_test_data = {}

i = 0
for key in jam_test_data.keys():
    data = {'uuid': jam_test_data[key]['uuid'],
            'longitude_latitude': jam_test_data[key]['longitude_latitude'],
            'length': jam_test_data[key]['length'],
            'numPts': float(jam_test_data[key]['numPts']),
            'speed': jam_test_data[key]['speed'],
            'startTime': jam_test_data[key]['startTime'],
            'loc_label': float(test_loc_label[i]),
            'length_label': float(test_length_label[i]),
            'numPts_label': float(test_numPts_label[i]),
            'speed_label': float(test_speed_label[i])}
    quantized_jam_test_data[key] = data
    i += 1
    
# Writing JSON data
with open(data_folder + 'quantized_jam_test_data.json', 'w') as json_file:
    json.dump(quantized_jam_test_data, json_file)